In [ ]:
#melhor redes ADAM normal

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

import tensorflow as tf
from tensorflow import keras
from keras import layers

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import numpy as np
from sklearn.metrics import recall_score, f1_score, confusion_matrix

epsilon = 1e-10
# Carregar o conjunto de dados
df = pd.read_csv('Dados_Trabalho_TP2.csv')

# Pré-processar os dados
le = LabelEncoder()
for column in df.columns:
    if df[column].dtype == object:
        df[column] = le.fit_transform(df[column])

# Dividir o conjunto de dados em recursos (X) e rótulos (y)
X = df.drop('Genero', axis=1)
y = df['Genero']

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


# Dividir o conjunto de dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# np.random.seed(0)
tf.random.set_seed(0)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

optimizers = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
initializers = ['uniform', 'lecun_uniform', 'normal', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']

acc_per_fold = []
loss_per_fold = []
f1score_per_fold = []
fold_no = 1
epochs_list = [10, 20, 30]
batch_sizes = [16, 32, 64]
results = []

for optimizer in optimizers:
    for initializer in initializers:
        for train, test in kfold.split(X_train, y_train):
            model3 = tf.keras.Sequential([
            tf.keras.Input(shape=(17,)),
            tf.keras.layers.Dense(units=64, activation="relu", kernel_initializer=initializer),
            tf.keras.layers.Dense(units=48, activation="relu", kernel_initializer=initializer),
            tf.keras.layers.Dense(units=32, activation="relu", kernel_initializer=initializer),
            tf.keras.layers.Dense(units=32, activation="relu", kernel_initializer=initializer),
            tf.keras.layers.Dense(units=16, activation="relu", kernel_initializer=initializer),
            tf.keras.layers.Dense(units=16, activation="relu", kernel_initializer=initializer),
            tf.keras.layers.Dense(units=16, activation="relu", kernel_initializer=initializer),
            tf.keras.layers.Dense(units=16, activation="relu", kernel_initializer=initializer),
            tf.keras.layers.Dense(units=16, activation="relu", kernel_initializer=initializer),
            tf.keras.layers.Dense(units=16, activation="relu", kernel_initializer=initializer),
            tf.keras.layers.Dense(units=16, activation="relu", kernel_initializer=initializer),
            tf.keras.layers.Dense(units=9, activation="softmax", kernel_initializer=initializer)
        ])

        model3.compile(optimizer=optimizer,
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])

        # Treina o modelo
        history4 = model3.fit(X_train, y_train, epochs=100, verbose=0)

        # Avalia o modelo com métricas de generalização
        scores3 = model3.evaluate(X_test, y_test, verbose=0)

        # Faz previsões no conjunto de teste
        y_pred = model3.predict(X_test)
        y_pred = np.argmax(y_pred, axis=1)  # Converte as previsões de one-hot para rótulos

        # Calcula Sensitivity (Recall), Specificity e F1
        sensitivity = recall_score(y_test, y_pred, average='macro')
        cm = confusion_matrix(y_test, y_pred)
        specificity = cm[0,0] / (cm[0,0] + cm[0,1] + epsilon)
        f1 = f1_score(y_test, y_pred, average='macro')

        # Armazena os resultados
        results.append({
            'optimizer': optimizer,
            'initializer': initializer,
            'accuracy': scores3[1] * 100,
            'loss': scores3[0],
            'sensitivity': sensitivity,
            'specificity': specificity,
            'f1': f1
        })

# Encontra o resultado com a maior acurácia
best_result = max(results, key=lambda x: x['accuracy'])

print('Melhor acurácia:', best_result['accuracy'])
print('Melhor otimizador:', best_result['optimizer'])
print('Melhor inicializador:', best_result['initializer'])
print('Sensibilidade:', best_result['sensitivity']*100)
print('Especificidade:', best_result['specificity']*100)
print('F1:', best_result['f1']*100)

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import accuracy_score, recall_score, f1_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pandas as pd
import numpy as np

# Carregar o conjunto de dados
df = pd.read_csv('Dados_Trabalho_TP2.csv')

# Pré-processar os dados
le = LabelEncoder()
for column in df.columns:
    if df[column].dtype == object:
        df[column] = le.fit_transform(df[column])

# Dividir o conjunto de dados em recursos (X) e rótulos (y)
X = df.drop('Genero', axis=1)
y = df['Genero']

# Normalizar os dados
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# Dividir o conjunto de dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir o número de folds
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Definindo o modelo SVM com kernel RBF
kernel = 'rbf'
svm = SVC(kernel=kernel)

# Ajuste de parâmetros utilizando GridSearchCV para evitar overfitting
param_grid = {'C': [0.1, 1, 10, 100, 1000], 'gamma': [10, 'scale', 'auto']}
grid = GridSearchCV(svm, param_grid, cv=kf, refit=True, scoring='accuracy', n_jobs=-1)
grid.fit(X, y)

best_model = grid.best_estimator_

# Avaliar o modelo usando k-fold cross validation
scores = []
sensitivities = []
specificities = []
f1_scores = []
epsilon = 1e-7  # para evitar divisão por zero

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    best_model.fit(X_train, y_train)
    y_pred = best_model.predict(X_test)
    fold_score = accuracy_score(y_test, y_pred)
    scores.append(fold_score)

    # Calcular a sensibilidade (recall), especificidade e F1 para este fold
    sensitivity = recall_score(y_test, y_pred, average='macro')
    cm = confusion_matrix(y_test, y_pred)
    specificity = cm[0,0] / (cm[0,0] + cm[0,1] + epsilon)
    f1 = f1_score(y_test, y_pred, average='macro')

    # Adicionar as métricas deste fold às listas correspondentes
    sensitivities.append(sensitivity)
    specificities.append(specificity)
    f1_scores.append(f1)

# Calcular a média e desvio padrão das acurácias
mean_score = np.mean(scores) * 100
std_score = np.std(scores)

# Calcular a média das sensibilidades, especificidades e F1
mean_sensitivity = np.mean(sensitivities)
mean_specificity = np.mean(specificities)
mean_f1 = np.mean(f1_scores)

# Armazenar os melhores resultados
best_result = {
    'accuracy': mean_score,
    'optimizer': grid.best_params_['C'],
    'initializer': grid.best_params_['gamma'],
    'sensitivity': mean_sensitivity,
    'specificity': mean_specificity,
    'f1': mean_f1
}

# Imprimir os melhores resultados
print('Melhor acurácia:', best_result['accuracy'])
print('Melhor otimizador:', best_result['optimizer'])
print('Melhor inicializador:', best_result['initializer'])
print('Sensibilidade:', best_result['sensitivity']*100)
print('Especificidade:', best_result['specificity']*100)
print('F1:', best_result['f1']*100)


In [ ]:
from scipy import stats
import numpy as np

# Pontuações do modelo SVM
scores_svm = scores

# Imprime as pontuações de acurácia do modelo SVM
print(f"Pontuações de acurácia do Modelo SVM: {scores_svm}")

# Extrai as pontuações de acurácia da lista 'results'
scores_model3 = [result['accuracy'] for result in results]

# Imprime as pontuações de acurácia do modelo 3
print(f"Pontuações de acurácia do Modelo 3: {scores_model3}")

# Realiza o teste t
t_stat_svm_3, p_val_svm_3 = stats.ttest_ind(scores_svm, scores_model3)

# Imprime os resultados
print(f"Modelo SVM vs Modelo 3: t statistic: {t_stat_svm_3:.2f}, p value: {p_val_svm_3:.12f}")

# Verifica se a diferença é significativa
if p_val_svm_3 < 0.05:
    print("Há uma diferença significativa no desempenho dos dois modelos.")
else:
    print("Não há uma diferença significativa no desempenho dos dois modelos.")

# Identifica o modelo com melhor desempenho
if np.mean(scores_svm) > np.mean(scores_model3):
    print("O Modelo SVM tem melhor desempenho.")
else:
    print("O Modelo 3 (Rede Neural) tem melhor desempenho.")